In [35]:
import tlsh
import random
import numpy
from scipy.spatial import distance

# Get the histogram of buckets

In [96]:
t1 = tlsh.hash(random.randbytes(1024))
ex = t1[8:]
ex

'4E00032499A67AF4923C7B035C846F6EACBD54946BD65950DABCE037D3C4AE22'

In [97]:
def to_hist(t):
    buckets = numpy.zeros(128, dtype=numpy.int32)
    for idx, (sixteen, one) in enumerate(zip(t[0::2], t[1::2])):
        buckets[4 * idx] = (int(sixteen,16) & 0x0C) >> 2
        buckets[4 * idx + 1] = (int(sixteen,16) & 0x03)
        buckets[4 * idx + 2] = (int(one,16) & 0x0C) >> 2
        buckets[4 * idx + 3] = (int(one,16) & 0x03)
    return buckets

In [98]:
buk = to_hist(ex)
buk

array([1, 0, 3, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0, 2, 1, 0, 2, 1, 2, 1, 2, 2,
       1, 2, 1, 3, 2, 2, 3, 3, 1, 0, 2, 1, 0, 2, 0, 3, 3, 0, 1, 3, 2, 3,
       0, 0, 0, 3, 1, 1, 3, 0, 2, 0, 1, 0, 1, 2, 3, 3, 1, 2, 3, 2, 2, 2,
       3, 0, 2, 3, 3, 1, 1, 1, 1, 0, 2, 1, 1, 0, 1, 2, 2, 3, 3, 1, 1, 2,
       1, 1, 2, 1, 1, 1, 0, 0, 3, 1, 2, 2, 2, 3, 3, 0, 3, 2, 0, 0, 0, 3,
       1, 3, 3, 1, 0, 3, 3, 0, 1, 0, 2, 2, 3, 2, 0, 2, 0, 2], dtype=int32)

# A Hamming approximation of TLSH distance

In [99]:
def hamming_three(buckets):
    ham = numpy.zeros(3 * 128, dtype=numpy.bool_)
    for idx, val in enumerate(buckets):
        if val == 3:
            ham[3 * idx + 2] = 1
        if val >= 2:
            ham[3 * idx + 1] = 1
        if val >= 1:
            ham[3 * idx] = 1
    return ham

In [100]:
hamming_three(buk)

array([ True, False, False, False, False, False,  True,  True,  True,
        True,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True,  True,  True,
       False, False, False,  True,  True, False,  True, False, False,
       False, False, False,  True,  True, False,  True, False, False,
        True,  True, False,  True, False, False,  True,  True, False,
        True,  True, False,  True, False, False,  True,  True, False,
        True, False, False,  True,  True,  True,  True,  True, False,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True, False, False, False, False, False,  True,  True, False,
        True, False, False, False, False, False,  True,  True, False,
       False, False, False,  True,  True,  True,  True,  True,  True,
       False, False, False,  True, False, False,  True,  True,  True,
        True,  True,

In [101]:
t2 = tlsh.hash(random.randbytes(1024))
ex2 = t2[8:]

In [102]:
tlsh.diff(t1,t2)

286

In [ ]:
buk2 = to_hist(ex2)

In [107]:
distance.hamming(hamming_three(buk),hamming_three(buk2)) * 3 * 128

171.0

# Another Hamming Distance, weighting extremes more

In [110]:
def hamming_five(buckets):
    ham = numpy.zeros(5 * 128, dtype=numpy.bool_)
    for idx, val in enumerate(buckets):
        if val == 3:
            ham[5 * idx + 4] = 1
            ham[5 * idx + 3] = 1
        if val >= 2:
            ham[5 * idx + 2] = 1
        if val < 1:
            ham[5 * idx + 1] = 1
            ham[5 * idx] = 1
    return ham

In [111]:
distance.hamming(hamming_five(buk),hamming_five(buk2)) * 5 * 128

275.0